In [1]:
import numpy as np
from scipy import ndimage
from PIL import Image
import matplotlib.pyplot as plt
import Ipynb_importer
import warp
import ransac

importing Jupyter notebook from warp.ipynb
importing Jupyter notebook from homography.ipynb


<font color = red>为什么用row在最后一步得不出正确结果，但是某些情况下却适用,和一维数组有关系吗？<font>

In [1]:
def normalize(points):
    """在齐次坐标意义下，对点集进行归一化，使最后一行的值为1"""
    #print(points)
    for i in range(points.shape[0]):
        #print(row)
        #print(points[-1])
        points[i] /= points[-1]
        #print(points[i])
        #print(points)
    #print('result:')
    #print(points,'\n')
    return points

In [3]:
def make_homog(points):
    """将点集(dim×的数组)转换为齐次坐标表示"""
    
    return np.vstack((points,np.ones((1,points.shape[1]))))

In [ ]:
def H_from_points(fp, tp):
    """ 使用线性DLT方法，计算单应性矩阵H，是fp映射到tp，点自动进行归一化 """

    if fp.shape != tp.shape:
        raise RuntimeError('number of points do not match')

    # 对点进行归一化
    # --映射起始点--
    m = np.mean(fp[:2], axis=1)
    maxstd = max(np.std(fp[:2], axis=1)) + 1e-9
    C1 = np.diag([1 / maxstd, 1 / maxstd, 1])
    C1[0][2] = -m[0] / maxstd
    C1[1][2] = -m[1] / maxstd
    fp = np.dot(C1, fp)

    # --映射起始点--
    m = np.mean(tp[:2], axis=1)
    maxstd = max(np.std(tp[:2], axis=1)) + 1e-9
    C2 = np.diag([1 / maxstd, 1 / maxstd, 1])
    C2[0][2] = -m[0] / maxstd
    C2[1][2] = -m[1] / maxstd
    tp = np.dot(C2, tp)

    # 创建用于线性方法的矩阵，对于每个对应对，矩阵中会出现两行数值
    nbr_correspondences = fp.shape[1]
    A = np.zeros((2 * nbr_correspondences, 9))
    for i in range(nbr_correspondences):
        A[2 * i] = [-fp[0][i], -fp[1][i], -1, 0, 0, 0,
                    tp[0][i] * fp[0][i], tp[0][i] * fp[1][i], tp[0][i]]
        A[2 * i + 1] = [0, 0, 0, -fp[0][i], -fp[1][i], -1,
                        tp[1][i] * fp[0][i], tp[1][i] * fp[1][i], tp[1][i]]

    U, S, V = np.linalg.svd(A)
    H = V[8].reshape((3, 3))

    # 反归一化
    H = np.dot(np.linalg.inv(C2), np.dot(H, C1))

    # 归一化并返回结果
    return H / H[2, 2]

In [15]:
def Haffine_from_points(fp,tp):
    """计算仿射变换H，使得tp是fp经过H得到"""
    
    if fp.shape != tp.shape:
        raise RuntimeError('Number of points do not match')
    
    #对点进行归一化
    #---映射起始点---
    m = np.mean(fp[:2],axis = 1)
    maxstd = np.max(np.std(fp[:2],axis = 1)) + 1e-9
    C1 = np.diag([1/maxstd,1/maxstd,1])
    C1[0][2] = -m[0]/maxstd
    C1[1][2] = -m[1]/maxstd
    fp_cond = np.dot(C1,fp)
    #print('fp_cond:\n',fp_cond)
    
    #---映射对应点---
    m = np.mean(tp[:2],axis = 1)
    C2 = C1.copy()
    C2[0][2] = -m[0]/maxstd
    C2[1][2] = -m[1]/maxstd
    tp_cond = np.dot(C2,tp)
    #print('tp_cond:\n',tp_cond)
    
    #因为归一化后的均值为0，所以平移量为0
    A = np.concatenate((fp_cond[:2],tp_cond[:2]),axis = 0)
    U,S,V = np.linalg.svd(A.T)
    #U,S,V = np.linalg.svd(A)
    
    #如Hartley和Zisserman著的Multiple View Geometry in Computer,Scond Ediotion所示
    #创建矩阵B和C
    tmp = V[:2].T
    B = tmp[:2]
    C = tmp[2:4]
    
    tmp2 = np.concatenate((np.dot(C,np.linalg.pinv(B)),np.zeros((2,1))),axis = 1)
    H = np.vstack((tmp2,[0,0,1]))
    
    #反归一化
    H = np.dot(np.linalg.inv(C2),np.dot(H,C1))
    
    return H/H[2,2]
    

In [16]:
def image_in_image(im1,im2,tp):
    """使用仿射变换im1放在im2上，使im1图像的角和tp尽可能地接近
    tp是其次表示的，并且是按照从左上角逆时针计算的"""
    
    #扭曲的点
    m,n = im1.shape[:2]
    fp = np.array([[0,m,m,0],[0,0,n,n],[1,1,1,1]])
    
    #计算仿射变换并将其应用于图像im1
    H = Haffine_from_points(tp,fp)
    im1_t = ndimage.affine_transform(im1,H[:2,:2],(H[0,2],H[1,2]),im2.shape[:2])
    #print(im1_t)
    alpha = (im1_t > 0)
    
    return (1-alpha)*im2 + alpha*im1_t

#仿射扭曲im1到im2
im1 = np.array(Image.open('beatles.jpg').convert('L'))
im2 = np.array(Image.open('billboard_for_rent.jpg').convert('L'))
#im2 = np.array(Image.open('beatles.jpg').convert('L'))

#选定一些目标点
tp = np.array([[264,358,540,264],[40,36,605,605],[1,1,1,1]])
#tp = np.array([[675,826,826,677],[55,52,281,277],[1,1,1,1]])

im3 = image_in_image(im1,im2,tp)

plt.figure()
plt.gray()
plt.imshow(im3)
plt.axis('equal')
plt.axis('off')
plt.show()

In [2]:
class RansacModel(object):
    """用于测试单应性矩阵的类"""
    
    def _init_(self,debug=False):
        self.debug=debug
        
    def fit(self,data):
        """计算选取的4个点对应的单应性矩阵"""
        
        #将其转置，来调用H_from_points()计算单应性矩阵
        data = data.T
        #print(data)
        #print('DATA  :one turn!!!!!!!!!!!!!!!!\n')
        
        #映射的起始点
        fp = data[:3,:4]
        #print(fp)
        #print('FP  :one turn!!!!!!!!!!!!!!!!\n')
        #映射的目标点
        tp = data[3:,:4]
        #print(tp)
        #print('TP  :one turn!!!!!!!!!!!!!!!!\n')
        
        #计算单应性矩阵，然后返回
        #print(H_from_points(fp,tp))
        #print('H:  @@@one turn!!!!!!!!!!!!!!!!\n')
        #return Haffine_from_points(fp,tp)
        return H_from_points(fp,tp)
        
    def get_error(self,data,H):
        """对所有的对应计算单应性矩阵，然后对每个变换的点，返回相应误差"""
        
        data = data.T
        
        #映射起始点
        fp = data[:3]
        #映射目标点
        tp = data[3:]
        
        #变换起始点
        fp_transformed = np.dot(H,fp)
        
        #归一化齐次坐标
        for i in range(3):
            fp_transformed[i] /= fp_transformed[2]
            
        #返回每个点的误差
        #print(np.sqrt(np.sum((tp-fp_transformed)**2, axis=0)))
        return np.sqrt(np.sum((tp-fp_transformed)**2, axis=0))
        
        

In [2]:
def H_from_ransac(fp,tp,model,maxiter=1000,match_theshold=10):
    """使用RANSAC稳健性估计点对应的单应性矩阵H
    fp,tp以齐次坐标表示"""
    
    #对应点对
    data = np.vstack((fp,tp))
    #print(data)
    #print('\n')
    
    #计算H，并返回
    H,ransac_data = ransac.ransac(data.T,model,4,maxiter,match_theshold,10,return_all=True)
    
    return H,ransac_data['inliers']